<a href="https://colab.research.google.com/github/GDjkhp/rvc-for-ai-music-covers/blob/main/RVC_Inference_by_GDjkhp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WTFPL

In [ ]:
#@title 0. Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 1. Install RVC
# taken from: https://colab.research.google.com/drive/1hmKPNeeReO4NHzOktJFMI2plYKy5F7ZI
# TODO: clean

# force py 3.10
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user

%cd /content
from IPython.display import clear_output
from ipywidgets import Button
import subprocess, shlex, os
from google.colab import drive

!git clone https://github.com/splendormagic/RVC_BahaaMahmoud /content/RVC

!pip install pip==24.0

!apt -y install -qq aria2
pretrains = ["f0D32k.pth","f0G32k.pth"]
new_pretrains = ["f0Ov2Super32kD.pth","f0Ov2Super32kG.pth"]

for file in pretrains:
    if not os.path.exists(f"/content/RVC/assets/pretrained_v2/{file}"):
        command = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/%s%s%s/resolve/main/pretrained_v2/%s -d /content/RVC/assets/pretrained_v2 -o %s" % ("Voice","Conversion","WebUI",file,file)
        try:
            subprocess.run(shlex.split(command))
        except Exception as e:
            print(e)

for file in new_pretrains:
    if not os.path.exists(f"/content/RVC/assets/pretrained_v2/{file}"):
        command = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/poiqazwsx/Ov2Super32kfix/resolve/main/%s -d /content/RVC/assets/pretrained_v2 -o %s" % (file,file)
        try:
            subprocess.run(shlex.split(command))
            print(shlex.split(command))
        except Exception as e:
            print(e)

!mkdir -p /content/dataset && mkdir -p /content/RVC/audios
!wget -nc https://raw.githubusercontent.com/RejektsAI/EasyTools/main/original -O /content/RVC/original.py
!wget -nc https://raw.githubusercontent.com/RejektsAI/EasyTools/main/app.py -O /content/RVC/demo.py
!wget -nc https://raw.githubusercontent.com/RejektsAI/EasyTools/main/easyfuncs.py -O /content/RVC/easyfuncs.py
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/download_files.py -O /content/RVC/download_files.py
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/a.png -O /content/RVC/a.png
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/easy_sync.py -O /content/RVC/easy_sync.py
!wget -nc https://huggingface.co/spaces/Rejekts/RVC_PlayGround/raw/main/app.py -O /content/RVC/playground.py
!wget -nc https://huggingface.co/spaces/Rejekts/RVC_PlayGround/raw/main/tools/useftools.py -O /content/RVC/tools/useftools.py
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/astronauts.mp3 -O /content/RVC/audios/astronauts.mp3
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/somegirl.mp3 -O /content/RVC/audios/somegirl.mp3
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/someguy.mp3 -O /content/RVC/audios/someguy.mp3
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/unchico.mp3 -O /content/RVC/audios/unchico.mp3
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/unachica.mp3 -O /content/RVC/audios/unachica.mp3
!cd /content/RVC && python /content/RVC/download_files.py

if not "installed" in locals():
    !cd /content/RVC && pip install -r requirements.txt
    !pip install mega.py gdown==5.1.0 pytube pydub gradio==3.42.0
installed=True

In [ ]:
#@title 2. Install Demucs (Optional)
#@markdown NOTE: If you want to split more components of a song (e.g. melody, piano, guitar, instrumental), use https://freemusicdemixer.com
!pip install demucs

In [ ]:
#@title 3. Demix Audio (Optional)
#@markdown NOTE: If you want to split more components of a song (e.g. melody, piano, guitar, instrumental), use https://freemusicdemixer.com
import os
working_dir = "/content/dataset" # @param {"type":"string"}
audio = "11. Gangsta Rap - Nia Nia Nia (Explicit).flac" # @param {"type":"string"}
!python -m demucs.separate "{working_dir}/{audio}" -o "{working_dir}"
stem_path = f"{working_dir}/htdemucs/{os.path.splitext(audio)[0]}"
!cp "{stem_path}/vocals.wav" "{working_dir}"
!cp "{stem_path}/bass.wav" "{working_dir}"
!cp "{stem_path}/drums.wav" "{working_dir}"
!cp "{stem_path}/other.wav" "{working_dir}"

In [ ]:
#@title 4. Infer
# Claude rewrote everything for simplicity (ongoing)
import os
import shutil
import requests
from pathlib import Path
import zipfile
from colorama import Fore
import subprocess
from urllib.parse import unquote

#@markdown # Settings
#@markdown ### Input/Output
#@markdown - Set current working directory. Input files must be here. Output files will be generated here.
working_dir = "/content/dataset" # @param {"type":"string"}
input_audio = "vocals.wav" # @param {"type":"string"}
output_audio = "momoi_output.wav" # @param {"type":"string"}
#@markdown ### Model
#@markdown - Only `Hugging Face` links are supported. **MAKE SURE** it's an `RVC` model and **NOT** a `so-vits-svc` model. It will return an error at download.
#@markdown - `RVC` models contain `*.pth and *.index` files while `so-vits-svc` models contain `config.json and *.pth` files.
model_url = "https://huggingface.co/LordDavis778/BlueArchivevoicemodels/resolve/main/SaibaMomoi.zip" # @param {"type":"string"}
decoded_url = unquote(model_url)
filename = os.path.basename(decoded_url)
model_name = os.path.splitext(filename)[0]
#@markdown ### Pitch
#@markdown - **Female -> Male : -12**
#@markdown - **Male -> Female : 12**
pitch = 12 # @param {"type":"slider","min":-12,"max":12,"step":1}
#@markdown ---
#@markdown ### Advanced
f0_method = "rmvpe" # @param ["rmvpe", "pm", "harvest"]
index_rate = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 0 #@param {type:"slider", min:0, max:1, step:0.01}
consonant_protection = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}

class RVCInference:
    def __init__(self, working_dir: str | Path):
        """
        Initialize RVC inference with a configurable working directory.

        Args:
            working_dir (str | Path): Path to the working directory for input/output files
        """
        self.input_files_dir = Path(working_dir)
        self.models = {}
        self._setup_directory_structure()

    def _setup_directory_structure(self):
        """Create necessary directory structure."""
        self.input_files_dir.mkdir(parents=True, exist_ok=True)

    def verify_files(self, paths: dict) -> None:
        """Verify all required input files exist."""
        # Verify all paths except output_path
        for key, path in paths.items():
            if key != 'output_path':  # Skip output path verification
                if not os.path.exists(path):
                    raise FileNotFoundError(f"{key} not found at: {path}")
                print(Fore.GREEN + f"Found {key} at: {path}")

        # Verify output directory exists
        output_dir = os.path.dirname(paths['output_path'])
        if not os.path.exists(output_dir):
            raise FileNotFoundError(f"Output directory not found at: {output_dir}")
        print(Fore.GREEN + f"Output will be saved to: {paths['output_path']}")

    def verify_input_file(self, input_filename: str) -> str:
        """Verify input file exists and return its full path."""
        input_path = self.input_files_dir / input_filename
        if not input_path.exists():
            raise FileNotFoundError(f"Input file not found: {input_path}")
        return str(input_path)

    def download_and_extract_model(self, url: str, model_name: str) -> bool:
        """Download and extract model from URL to appropriate directory."""
        model_dir = self.input_files_dir / model_name
        model_dir.mkdir(exist_ok=True)

        try:
            # Download zip file
            print(f"Downloading model from {url}")
            response = requests.get(url, stream=True)
            response.raise_for_status()

            # Save zip file
            zip_path = model_dir / "temp.zip"
            with open(zip_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            # Extract zip file
            print(f"Extracting model to {model_dir}")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(model_dir)

            # Clean up zip file
            zip_path.unlink()

            # Register model
            self._register_model(model_name, model_dir)
            return True

        except Exception as e:
            print(f"Error downloading/extracting model: {e}")
            if model_dir.exists():
                shutil.rmtree(model_dir)
            return False

    def _register_model(self, model_name: str, model_dir: Path):
        """Register model paths after download/extraction."""
        # Find .pth and .index files
        pth_files = list(model_dir.rglob("*.pth"))
        index_files = list(model_dir.rglob("*.index"))

        if not pth_files or not index_files:
            raise FileNotFoundError(f"Missing required files in {model_dir}")

        self.models[model_name] = {
            'pth': str(pth_files[0]),
            'index': str(index_files[0])
        }

    def setup_paths(self, model_name: str, input_filename: str) -> dict:
        """Setup all necessary paths for inference."""
        if model_name not in self.models:
            raise ValueError(f"Model {model_name} not found. Please download it first.")

        return {
            'model_path': self.models[model_name]['pth'],
            'index_path': self.models[model_name]['index'],
            'input_path': self.verify_input_file(input_filename),
            'output_path': str(self.input_files_dir)
        }

    def set_output_filename(self, filename: str):
        """Set custom output filename and remove if exists."""
        output_path = self.input_files_dir / filename
        if output_path.exists():
            output_path.unlink()
        return str(output_path)

    def run_inference(self, model_name: str, input_filename: str = "vocals.wav",
                     output_filename: str = "output.wav", pitch: int = 0,
                     f0_method: str = "rmvpe", index_rate: float = 0.5,
                     volume_norm: float = 0, consonant_protect: float = 0.5) -> str:
        """Run the RVC inference pipeline."""
        # Setup and verify paths
        paths = self.setup_paths(model_name, input_filename)
        paths['output_path'] = self.set_output_filename(output_filename)
        self.verify_files(paths)

        # Set environment variables
        os.environ['index_root'] = os.path.dirname(paths['index_path'])
        os.environ['weight_root'] = os.path.dirname(paths['model_path'])

        # Prepare command
        input = paths['input_path']
        index = os.path.basename(paths['index_path'])
        output = paths['output_path']
        model = os.path.basename(paths['model_path'])

        # Run command
        os.chdir("/content/RVC")
        command = [
            "python", "tools/cmd/infer_cli.py",
            "--f0up_key", str(pitch),
            "--input_path", input,
            "--index_path", index,
            "--f0method", f0_method,
            "--opt_path", output,
            "--model_name", model,
            "--index_rate", str(index_rate),
            "--device", "cuda:0",
            "--is_half", "True",
            "--filter_radius", "3",
            "--resample_sr", "0",
            "--rms_mix_rate", str(volume_norm),
            "--protect", str(consonant_protect)
        ]
        result = subprocess.run(command, capture_output=True, text=True)

        if result.returncode == 0:
            print(result.stdout)
        else:
            print("Error occurred:", result.stderr)

        return paths['output_path']

def run_voice_conversion(
    model_url: str,
    model_name: str,
    working_dir: str | Path = "/content/workingdir",
    input_filename: str = "vocals.wav",
    output_filename: str = "output.wav",
    pitch: int = 0,
    f0_method: str = "rmvpe",
    index_rate: float = 0.5,
    volume_norm: float = 0,
    consonant_protect: float = 0.5
):
    """
    Main function to run voice conversion with configurable working directory.

    Args:
        model_url (str): URL to download the model from
        model_name (str): Name to identify the model
        working_dir (str | Path): Directory for input/output files and models
        input_filename (str): Name of the input audio file
        output_filename (str): Name for the output audio file
        pitch (int): Pitch adjustment value (-12 to 12)
    """
    rvc = RVCInference(working_dir=working_dir)

    # Download and extract model if not already present
    if model_name not in rvc.models:
        success = rvc.download_and_extract_model(model_url, model_name)
        if not success:
            raise RuntimeError(f"Failed to download/extract model {model_name}")

    # Run inference with default parameters
    output_path = rvc.run_inference(
        model_name=model_name,
        input_filename=input_filename,
        output_filename=output_filename,
        pitch=pitch,
        f0_method=f0_method,
        index_rate=index_rate,
        volume_norm=volume_norm,
        consonant_protect=consonant_protect
    )

# Example usage:
if __name__ == "__main__":
    run_voice_conversion(
        model_url=model_url,
        model_name=model_name,
        working_dir=working_dir,
        input_filename=input_audio,
        output_filename=output_audio,
        pitch=pitch,
        f0_method=f0_method,
        index_rate=index_rate,
        volume_norm=volume_normalization,
        consonant_protect=consonant_protection
    )